In [1]:
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, FalseNegatives, FalsePositives, TrueNegatives, \
    TruePositives, AUC
from tensorflow.keras.losses import BinaryCrossentropy
import os
import math
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Flatten, Dropout,BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint

2023-06-24 22:45:41.924069: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-24 22:45:42.375718: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17939073461060000280
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5874319360
locality {
  bus_id: 1
  links {
  }
}
incarnation: 716881128996807693
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:2b:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


2023-06-24 22:45:42.959280: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-24 22:45:42.971944: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-24 22:45:42.972076: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-06-24 22:45:43.690175: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-24 22:45:43.690343: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-24 22:45:43.690430: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
# Set the GPU device
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    tf.config.experimental.set_visible_devices(physical_devices[0], 'GPU')

In [5]:
# Set GPU device
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Replace '0' with the GPU device ID

In [6]:
# Set seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [7]:
# Define your data generator
class CustomImageDataGenerator(tf.keras.preprocessing.image.ImageDataGenerator):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def flow_from_directory(self, *args, **kwargs):
        iterator = super().flow_from_directory(*args, **kwargs)
        while True:
            try:
                data = next(iterator)
                yield data
            except (OSError, tf.errors.InvalidArgumentError):
                continue

In [7]:
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 8


In [8]:

img_data_gn = CustomImageDataGenerator(rescale=1./255,rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [8]:
img_data_gn = ImageDataGenerator(rescale=1./255)

In [9]:
train_path = "/media/maged/New Volume/NU/1-Spring 23/Grad Project 2/datasets again/Defacto + others/train"
valid_path = "/media/maged/New Volume/NU/1-Spring 23/Grad Project 2/datasets again/Defacto + others/validation"

In [11]:
train_batches = img_data_gn.flow_from_directory(train_path,
                                                target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                class_mode='binary',
                                                classes=['authentic', 'tampered'],
                                                batch_size=BATCH_SIZE,
                                                shuffle=True)

valid_batches = img_data_gn.flow_from_directory(valid_path,
                                                target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                class_mode='binary',
                                                classes=['authentic', 'tampered'],
                                                batch_size=BATCH_SIZE, shuffle=True)

Found 20800 images belonging to 2 classes.
Found 8443 images belonging to 2 classes.


In [11]:
#train_batches.class_indices

In [12]:

# Get the number of images in the training and validation sets
train_image_count = sum([len(files) for _, _, files in os.walk(train_path)])
valid_image_count = sum([len(files) for _, _, files in os.walk(valid_path)])



# Calculate the steps per epoch based on the batch size and number of images
steps_per_epoch = math.ceil(train_image_count / BATCH_SIZE)
validation_steps = math.ceil(valid_image_count / BATCH_SIZE)

In [13]:
model = ResNet50(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), weights= None , include_top=False)


2023-06-24 22:46:31.078352: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-24 22:46:31.078732: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-24 22:46:31.079041: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [14]:
# for layer in model.layers:
#     layer.trainable = False

In [14]:
last_layer = model.get_layer('conv5_block3_out')
last_output = last_layer.output

In [15]:
x = layers.Flatten()(last_output)
x = layers.Dense(1000, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = layers.Dense(1000, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(500, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x) 
x = Dense(500, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x) 
x = layers.Dense(1, activation='sigmoid')(x)

In [16]:
from tensorflow import keras

In [17]:
newModel = Model(model.input, x)

In [18]:
newModel.compile(Adam(learning_rate=0.001), loss=BinaryCrossentropy(),
                 metrics=[AUC(), BinaryAccuracy(), Precision(), Recall(), FalseNegatives(),
                          FalsePositives(), TrueNegatives(), TruePositives() ])

In [19]:
newModel.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [38]:
# file_path = 'Experiment_8_2.h5'
# callbacks = [
#     ModelCheckpoint(file_path, monitor='val_loss', mode='min', save_best_only=True, verbose=1),
# ]

In [39]:
#newModel.loadlt of `train_function` (Emptyweights(file_path)

In [20]:
# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.0000001, verbose=1)

In [24]:
checkpoint_path = '/home/maged/models/model_{epoch:02d}_accuracy_{accuracy:.4f}.h5'

In [24]:
checkpoint_callback = ModelCheckpoint(
    checkpoint_path,
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)

In [21]:
checkpoint_path = '/home/maged/Final_model_Resnet50_weights.h5'

In [22]:
newModel.load_weights(checkpoint_path)

ValueError: Layer count mismatch when loading weights from file. Model expected 115 layers, found 106 saved layers.

In [27]:
#class_weights_dict = {0: 1, 1: 3} 
history1 = newModel.fit(train_batches, verbose=1, validation_data=valid_batches,
                         epochs=50, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps,
                         callbacks=[early_stopping, reduce_lr])

2023-06-25 00:21:32.183176: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 1/50
2600/2600 [==============================] - ETA: 0s - loss: 0.6836 - auc_1: 0.5931 - binary_accuracy: 0.5656 - precision_1: 0.5549 - recall_1: 0.5398 - false_negatives_1: 4656.0000 - false_positives_1: 4380.0000 - true_negatives_1: 6303.0000 - true_positives_1: 5461.0000

2023-06-25 00:27:36.766636: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


2600/2600 [==============================] - 417s 87ms/step - loss: 0.6836 - auc_1: 0.5931 - binary_accuracy: 0.5656 - precision_1: 0.5549 - recall_1: 0.5398 - false_negatives_1: 4656.0000 - false_positives_1: 4380.0000 - true_negatives_1: 6303.0000 - true_positives_1: 5461.0000 - val_loss: 0.6899 - val_auc_1: 0.4763 - val_binary_accuracy: 0.5554 - val_precision_1: 0.5452 - val_recall_1: 0.9292 - val_false_negatives_1: 314.0000 - val_false_positives_1: 3440.0000 - val_true_negatives_1: 565.0000 - val_true_positives_1: 4124.0000 - lr: 0.0010
Epoch 2/50
2600/2600 [==============================] - 225s 86ms/step - loss: 0.6793 - auc_1: 0.6035 - binary_accuracy: 0.5719 - precision_1: 0.5593 - recall_1: 0.5657 - false_negatives_1: 4394.0000 - false_positives_1: 4510.0000 - true_negatives_1: 6173.0000 - true_positives_1: 5723.0000 - val_loss: 0.6591 - val_auc_1: 0.6860 - val_binary_accuracy: 0.6340 - val_precision_1: 0.6294 - val_recall_1: 0.7386 - val_false_negatives_1: 1160.0000 - val_fal

In [28]:
newModel.save('Final_model_Resnet50 copy_move.h5')

In [23]:
newModel.save('Final_model_Resnet50 again.json')

2023-06-17 22:26:26.324645: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1000]
	 [[{{node inputs}}]]
2023-06-17 22:26:26.340290: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,1000]
	 [[{{node inputs}}]]
2023-06-17 22:26:26.352628: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,500]
	 [[{{node inputs}}]]
2023-06-17 22:

INFO:tensorflow:Assets written to: Final_model_Resnet50 again.json/assets


INFO:tensorflow:Assets written to: Final_model_Resnet50 again.json/assets


In [24]:
newModel.save_weights('Final_model_Resnet50_weights again.h5')

In [25]:
from tensorflow.keras.models import model_from_json

In [54]:
# load json and create model
json_file = open('CNNImageSplicingDetectorModelResNet50_2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

FileNotFoundError: [Errno 2] No such file or directory: 'CNNImageSplicingDetectorModelResNet50_2.json'

In [29]:
test_path = "/media/maged/New Volume/NU/1-Spring 23/Grad Project 2/datasets again/Defacto + others/test"
test_batches = img_data_gn.flow_from_directory(test_path,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH),
                                               class_mode='binary',
                                               classes=['authentic', 'tampered'],
                                               batch_size=240)

Found 7311 images belonging to 2 classes.


In [30]:
test_image_count = sum([len(files) for _, _, files in os.walk(test_path)])
test_steps = math.ceil(test_image_count / BATCH_SIZE)


In [31]:
newModel.compile(Adam(learning_rate=0.001), loss=BinaryCrossentropy(),
                 metrics=[AUC(), BinaryAccuracy(), Precision(), Recall(), FalseNegatives(),
                          FalsePositives(), TrueNegatives(), TruePositives() ])

In [32]:
test_loss, test_accuracy = newModel.evaluate(test_batches, steps=test_steps)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

2023-06-25 00:59:03.104939: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-25 00:59:05.667650: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 896.33MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-06-25 00:59:05.832688: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 896.33MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-06-25 00:59:05.937978: W tensorflow/tsl/framework/bfc_alloc

914/914 [==============================] - 201s 215ms/step - loss: 0.7685 - auc_2: 0.7056 - binary_accuracy: 0.6180 - precision_2: 0.7240 - recall_2: 0.3757 - false_negatives_2: 2272.0000 - false_positives_2: 521.0000 - true_negatives_2: 3151.0000 - true_positives_2: 1367.0000


ValueError: too many values to unpack (expected 2)

In [17]:
import h5py

with h5py.File('/home/maged/ResNet50_3.h5', 'r') as f:
    print(list(f.keys()))  # Print the keys of groups and datasets


['model_weights', 'optimizer_weights']


In [55]:
model_json = newModel.to_json()


In [18]:
with open("CNNImageSplicingDetectorModelResNet50_4k.json", "w") as json_file:
    json_file.write(model_json)

NameError: name 'model_json' is not defined

In [ ]:
for steps in range(len(test_batches)):
    test_images, test_labels = next(test_batches)
    (newModel.evaluate(test_images, test_labels))


In [ ]:

#serialize weights to HDF5
newModel.save_weights("ResNet50.h5")


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:

um_test_samples = len(test_batches.labels)

# Generate predictions for the test data using the model
y_pred = model.predict(test_batches)
y_pred_classes = np.argmax(y_pred, axis=1)

# Get the true labels from the test data generator
y_true = test_batches.labels

# Generate the confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

# Plot the confusion matrix
class_labels = test_batches.class_indices.keys()
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d',
            xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()